In [1]:
# !pip install focal-loss-torch

In [2]:
import torch
from models.models import *
device = 'cuda' if torch.cuda.is_available() else 'cpu'
device

/home/server-ailab-12gb/miniconda3/envs/Khoa_env/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


'cuda'

In [3]:
architecture = "TSF"
# architecture = "TSF_hard_voting"
# architecture = "TSF_Resnet_Swin"
# architecture = "TSF_ViT_Resnet"
# architecture = "TSF_ViT_Swin"
# architecture = "Baseline"
# architecture = "Baseline_hard_voting"
# architecture = "Baseline_Resnet_Swin"
# architecture = "Baseline_ViT_Resnet"
# architecture = "Baseline_ViT_Swin"
# architecture = "TSF_Resnet"
# architecture = "TSF_ViT"
# architecture = "TSF_Swin"
train_index_checkpoint = '30'
test_index_checkpoint = ''
gamma = 2
alpha = 0.3
version = architecture + "_" + str(gamma) + "_" + str(alpha)

In [4]:
# FOLDER = "/home/server-ailab-12gb/DUC-MMM"
FOLDER = "/home/server-ailab-12gb/DUC-MMM"
data_dir = '/home/server-ailab-12gb/DUC-MMM/data/Musti'
train_dir = os.path.join(data_dir, 'train')
test_dir = os.path.join(data_dir, 'test')
train_img_dir = os.path.join(train_dir, 'img')
train_file_path = os.path.join(train_dir, 'train.json')

test_img_dir = os.path.join(test_dir, 'img')
test_file_path = os.path.join(test_dir, 'test.json')
history_save_folder = os.path.join(FOLDER, 'history_save')
checkpoint_dir = os.path.join(FOLDER, 'checkpoint')
checkpoint_directory = os.path.join(checkpoint_dir, version)

predict_dir = os.path.join(FOLDER, "predict")
predict_path = os.path.join(predict_dir, version + '.json')
report_dir = os.path.join(FOLDER, "report")
report_path = os.path.join(report_dir, version)

report_dir = os.path.join(FOLDER, "report")
report_path = os.path.join(report_dir, version)
if os.path.exists(history_save_folder)==False:
    os.mkdir(history_save_folder) 
if os.path.exists(checkpoint_directory)==False:
    os.mkdir(checkpoint_directory) 
if os.path.exists(report_path)==False:
    os.mkdir(report_path) 
lr = 0.001
epoch = 0
loss = 10000

In [5]:
image_processor = AutoImageProcessor.from_pretrained("google/vit-base-patch16-224-in21k")
tokenizer = BertTokenizer.from_pretrained('bert-base-multilingual-cased')
vit_encoder = ViTModel.from_pretrained("google/vit-base-patch16-224-in21k")
resnet_encoder = ResNetForImageClassification.from_pretrained("microsoft/resnet-50")
resnet_encoder = nn.Sequential(*list(resnet_encoder.children())[:-1])
swintransformer = AutoModelForImageClassification.from_pretrained("microsoft/swin-tiny-patch4-window7-224")
swintransformer = nn.Sequential(*list(swintransformer.children())[:-1])
text_encoder = BertModel.from_pretrained('bert-base-multilingual-cased')
encoder_layer = text_encoder.encoder
text_embedding= text_encoder.embeddings
pooling_layer= text_encoder.pooler

In [6]:
early_stopper = EarlyStopper(patience=5, min_delta=0)
dataset_train = Dataset_MMM(train_file_path, train_img_dir,image_processor,tokenizer)
dataloader_train = DataLoader(dataset_train,batch_size=20,drop_last=True, shuffle=True)

/home/server-ailab-12gb/DUC-MMM/data/Musti/train/img
Len: 2374


In [7]:
img_encoder =[vit_encoder,resnet_encoder,swintransformer]
model = get_model(architecture, img_encoder,encoder_layer,text_embedding, pooling_layer)

## Train

In [8]:

model = model.to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=lr)

if len(train_index_checkpoint) > 0:
    PATH = os.path.join(checkpoint_directory, train_index_checkpoint + CHECKPOINT_EXTENSION)
    print("Load checkpoint: "+str(PATH))
    epoch, model, optimizer, loss = load_checkpoint(PATH, model, optimizer)

history_losses = train(model=model,
                       train_loader=dataloader_train,
                       device=device ,
                       epochs=100, 
                       optimizer=optimizer, 
                       cur_epoch=epoch, 
                       best_valid_loss=loss,
                       gamma=gamma,
                       alpha=alpha,
                       checkpoint_directory=checkpoint_directory,
                       report_path=report_path,
                       early_stopper=early_stopper)
write_json(history_losses, parient_dir=history_save_folder, name=version)

Load checkpoint: /home/server-ailab-12gb/DUC-MMM/checkpoint/TSF_2_0.3/30.pt


Epoch 31: 100%|██████████| 118/118 [01:49<00:00,  1.08it/s, loss=0.0766]


Accuracy: 0.8508474738416025


Epoch 32: 100%|██████████| 118/118 [01:49<00:00,  1.08it/s, loss=0.0775]


Accuracy: 0.8411017085536051


...

Accuracy: 0.7521186577061475


Epoch 35: 100%|██████████| 118/118 [01:44<00:00,  1.13it/s, loss=0.131]


Accuracy: 0.764830521102679


Epoch 36: 100%|██████████| 118/118 [01:44<00:00,  1.13it/s, loss=0.122]

Accuracy: 0.7766949281854144
Early stop at epoch: 36 with valid loss: 0.1221879826763929
